<a href="https://colab.research.google.com/github/fgarcia26/Advanced-Learning---FGV-RI/blob/main/FinalProject_AdvancedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script - NightLights and Economic Growth

### Final Project - Advanced Learning 2022.2 

Fernanda Garcia & Julia Duó 

\\

> ***Does night time satellite imagery predict GDP per capita distribution in São Paulo region?***

\\


### Data & Methodology 

Data comes from: 


*   NASA: https://www.ngdc.noaa.gov/eog/dmsp/downloadV4composites.html
*   IBGE: https://www.ibge.gov.br/estatisticas/sociais/populacao22827-censo-2020-censo4.html?=t=downloads

\\

### Outline



1.   Download the necessary packages
2.   Library
3. Download the datasets
4. Following the CNN code used by META











## 1. Download the necessary packages

In [ ]:
! pip install wget
! pip install gdal
! pip install retrying
#! pip install cStringIO --> This package was DISCONTINUED (now, use IO)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=71709347cc81db7853e2e3b518ea2f1f862cbdfef5ac564de9139fc13943e669
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# We first tried using the basedosdados scraping of the IBGE dataset, but it 
# did not work

#! pip install basedosdados

## 2. Libraries 

In [ ]:
import pandas as pd
import numpy as np
#import basedosdados as bd
import wget
import time
import os
import os.path
from osgeo import gdal, ogr, osr
from scipy import ndimage
from scipy import misc
import io
gdal.UseExceptions()
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline
import urllib
import pandas as pd
import numpy as np

## 3. Download the datasets

In our case, there is an image and a dataset.

In [ ]:
# First, we will load the image directly from the NASA website.

night_image_url = 'https://www.ngdc.noaa.gov/eog/data/web_data/v4avg_lights_x_pct/F152000.v4b.avg_lights_x_pct.tgz'
wget.download(night_image_url)

'F152000.v4b.avg_lights_x_pct.tgz'

In [ ]:
# Second, we will load the dataset from our personal device.

# One possibility is to upload the dataset in github and then download 
# directly from there.

from google.colab import files
uploaded = files.upload()

income_df = pd.read_excel((uploaded['/Users/fernandagarcia/Downloads/Agregado_de_setores_2000_SP_RM/S∆o Paulo1/Responsavel1_SP1.XLS']))

Saving Responsavel1_SP1.XLS to Responsavel1_SP1.XLS


KeyError: ignored

## 4. Following the CNN code used by META

The CNN code used by META can be found in this link: https://dataforgood.facebook.com/dfg/docs/methodology-high-resolution-population-density-maps

In [ ]:
# Following the META code 

# First, we could create clusters within the IBGE database. 
# We do not need to create the wealth variable, for instance, 
# because our dataset already emcompasses the average of the 
# income in the unit of observation. 

# Moreover, the IBGE dataset is already divided within the 
# unit of observation that we are interested in: setor censitário.

# The problem here is that, even though we could clusterize the 
# IBGE, the ShapeFile is not working to subset the data according 
# to the geolocation. 

cluster_all = []
wealth_all = []
with open(income_url) as f:
    for line in f:
        cluster = int(line[28:736])
        wealth = int(column[2])
        cluster_all.append(cluster)
        wealth_all.append(wealth)

df = pd.DataFrame({'cluster': cluster_all, 'wlthindf': wealth_all})
cluster_avg_asset = df.groupby('cluster')['wlthindf'].median().reset_index()

df_location = pd.read_csv(cluster_file)[['DHSCLUST', 'LATNUM', 'LONGNUM']]
result = cluster_avg_asset.merge(df_location, how='inner', left_on='cluster', right_on='DHSCLUST')[['cluster', 'wlthindf', 'LATNUM', 'LONGNUM']]
result.rename(columns={'LATNUM': 'latitude', 'LONGNUM':'longitude'}, inplace=True)
result.to_csv('intermediate_files/rwanda_cluster_avg_asset_2010.csv', index=False)

In [ ]:
# Now, we will use the RASTER function to read our image files

def read_raster(raster_file):
    """
    Function
    --------
    read_raster

    Given a raster file, get the pixel size, pixel location, and pixel value

    Parameters
    ----------
    raster_file : string
        Path to the raster file

    Returns
    -------
    x_size : float
        Pixel size
    top_left_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the top-left point in each pixel
    top_left_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the top-left point in each pixel
    centroid_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the centroid in each pixel
    centroid_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the centroid in each pixel
    bands_data : numpy.ndarray  shape: (number of rows, number of columns, 1)
        Pixel value
    """
    raster_dataset = gdal.Open(raster_file, gdal.GA_ReadOnly)
    # get project coordination
    proj = raster_dataset.GetProjectionRef()
    bands_data = []
    # Loop through all raster bands
    for b in range(1, raster_dataset.RasterCount + 1):
        band = raster_dataset.GetRasterBand(b)
        bands_data.append(band.ReadAsArray())
        no_data_value = band.GetNoDataValue()
    bands_data = np.dstack(bands_data)
    rows, cols, n_bands = bands_data.shape

    # Get the metadata of the raster
    geo_transform = raster_dataset.GetGeoTransform()
    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = geo_transform
    
    # Get location of each pixel
    x_size = 1.0 / int(round(1 / float(x_size)))
    y_size = - x_size
    y_index = np.arange(bands_data.shape[0])
    x_index = np.arange(bands_data.shape[1])
    top_left_x_coords = upper_left_x + x_index * x_size
    top_left_y_coords = upper_left_y + y_index * y_size
    # Add half of the cell size to get the centroid of the cell
    centroid_x_coords = top_left_x_coords + (x_size / 2)
    centroid_y_coords = top_left_y_coords + (y_size / 2)

    return (x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data)


# Helper function to get the pixel index of the point
def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    """
    Function
    --------
    get_cell_idx

    Given a point location and all the pixel locations of the raster file,
    get the column and row index of the point in the raster

    Parameters
    ----------
    lon : float
        Longitude of the point
    lat : float
        Latitude of the point
    top_left_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the top-left point in each pixel
    top_left_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the top-left point in each pixel
    
    Returns
    -------
    lon_idx : int
        Column index
    lat_idx : int
        Row index
    """
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx

# The code worked! 

In [ ]:
# Creating the raster file based on our image

raster_file = 'https://www.ngdc.noaa.gov/eog/data/web_data/v4avg_lights_x_pct/F152000.v4b.avg_lights_x_pct.tgz'
x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data = read_raster(raster_file)

# save the result in compressed format - see https://docs.scipy.org/doc/numpy/reference/generated/numpy.savez.html
np.savez('intermediate_files/nightlight.npz', top_left_x_coords=top_left_x_coords, top_left_y_coords=top_left_y_coords, bands_data=bands_data)

RuntimeError: ignored

In [ ]:
def get_shp_extent(shp_file):
    """
    Function
    --------
    get_shp_extent

    Given a shapefile, get the extent (boundaries)

    Parameters
    ----------
    shp_file : string
        Path to the shapefile
    
    Returns
    -------
    extent : tuple
        Boundary location of the shapefile (x_min, x_max, y_min, y_max)
    """
    inDriver = ogr.GetDriverByName("ESRI Shapefile")
    inDataSource = inDriver.Open(inShapefile, 0)
    inLayer = inDataSource.GetLayer()
    extent = inLayer.GetExtent()
    x_min_shp, x_max_shp, y_min_shp, y_max_shp = extent
    return extent

In [ ]:
# Helper functions to download images from Google Maps API

from retrying import retry

@retry(wait_exponential_multiplier=1000, wait_exponential_max=3600000)
def save_img(url, file_path, file_name):
    """
    Function
    --------
    save_img

    Given a url of the map, save the image

    Parameters
    ----------
    url : string
        URL of the map from Google Map Static API
    file_path : string
        Folder name of the map
    file_name : string
        File name
    
    Returns
    -------
    None
    """
    a = urllib.urlopen(url).read()
    b = io.StringIO(a)
    image = ndimage.imread(b, mode='RGB')
    # when no image exists, api will return an image with the same color. 
    # and in the center of the image, it said'Sorry. We have no imagery here'.
    # we should drop these images if large area of the image has the same color.
    if np.array_equal(image[:,:10,:],image[:,10:20,:]):
        pass
    else:
        misc.imsave(file_path + file_name, image[50:450, :, :])

# Now read in the shapefile for Rwanda and extract the edges of the country
inShapefile = "/Users/fernandagarcia/Downloads/3550308/3550308.SHP"
x_min_shp, x_max_shp, y_min_shp, y_max_shp = get_shp_extent(inShapefile)

left_idx, top_idx = get_cell_idx(x_min_shp, y_max_shp, top_left_x_coords, top_left_y_coords)
right_idx, bottom_idx = get_cell_idx(x_max_shp, y_min_shp, top_left_x_coords, top_left_y_coords)

key = 'YOUR_GOOGLE_MAP_API_KEY'
m = 1
for i in xrange(left_idx, right_idx + 1):
    for j in xrange(top_idx, bottom_idx + 1):
        lon = centroid_x_coords[i]
        lat = centroid_y_coords[j]
        url = 'https://maps.googleapis.com/maps/api/staticmap?center=' + str(lat) + ',' + \
               str(lon) + '&zoom=16&size=400x500&maptype=satellite&key=' + key
        lightness = bands_data[j, i, 0]
        file_path = 'google_image/' + str(lightness) + '/'
        if not os.path.isdir(file_path):
            os.makedirs(file_path)
        file_name = str(i) + '_' + str(j) +'.jpg'
        save_img(url, file_path, file_name)
        if m % 100 == 0:
            print (m)
        m += 1

AttributeError: ignored